### About The Datasets

The dataset consists of 10 000 data points stored as rows with 14 features in columns
- **UID:** unique identifier ranging from 1 to 10000
- **product ID:** consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number
- **air temperature [K]:** generated using a random walk process later normalized to a standard deviation of 2 K around 300 K
- **process temperature [K]:** generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.
- **rotational speed [rpm]:** calculated from a power of 2860 W, overlaid with a normally distributed noise
- **torque [Nm]:** torque values are normally distributed around 40 Nm with a Ïƒ = 10 Nm and no negative values.
- **tool wear [min]:** The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process. and a

'machine failure' label that indicates, whether the machine has failed in this particular datapoint for any of the following failure modes are true.

The machine failure consists of five independent failure modes

- **Tool wear failure (TWF):** the tool will be replaced of fail at a randomly selected tool wear time between 200 â€“ 240 mins (120 times in our dataset). At this point in time, the tool is replaced 69 times, and fails 51 times (randomly assigned).
- **Heat dissipation failure (HDF):** heat dissipation causes a process failure, if the difference between air- and process temperature is below 8.6 K and the toolâ€™s rotational speed is below 1380 rpm. This is the case for 115 data points.
- **Power failure (PWF):** the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset.
- **Overstrain failure (OSF):** if the product of tool wear and torque exceeds 11,000 minNm for the L product variant (12,000 M, 13,000 H), the process fails due to overstrain. This is true for 98 datapoints.
- **Random failures (RNF):** each process has a chance of 0,1 % to fail regardless of its process parameters. This is the case for only 5 datapoints, less than could be expected for 10,000 datapoints in our dataset.

If at least one of the above failure modes is true, the process fails and the 'machine failure' label is set to 1. It is therefore not transparent to the machine learning method, which of the failure modes has caused the process to fail

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [28]:
from pandas_profiling import ProfileReport

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

### Data Descriptive Analysis

In [6]:
df = pd.read_csv('data.csv')

In [7]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [9]:
df.tail()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0
9999,10000,M24859,M,299.0,308.7,1500,40.2,30,0,0,0,0,0,0


In [10]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [12]:
df.shape

(10000, 14)

In [13]:
df.dtypes

UDI                          int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
dtype: object

In [14]:
df.nunique()

UDI                        10000
Product ID                 10000
Type                           3
Air temperature [K]           93
Process temperature [K]       82
Rotational speed [rpm]       941
Torque [Nm]                  577
Tool wear [min]              246
Machine failure                2
TWF                            2
HDF                            2
PWF                            2
OSF                            2
RNF                            2
dtype: int64

In [15]:
df.duplicated().sum()

0

In [16]:
df.isnull().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Machine failure          10000 non-null  int64  
 9   TWF                      10000 non-null  int64  
 10  HDF                      10000 non-null  int64  
 11  PWF                      10000 non-null  int64  
 12  OSF                      10000 non-null  int64  
 13  RNF                      10000 non-null  int64  
dtypes: float64(3), int64(9)

In [18]:
round(df.describe(),4)

,UDI,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000
mean,5000.5000,300.0049,310.0056,1538.7761,39.9869,107.9510,0.0339,0.0046,0.0115,0.0095,0.0098,0.0019
std,2886.8957,2.0003,1.4837,179.2841,9.9689,63.6541,0.1810,0.0677,0.1066,0.0970,0.0985,0.0435
min,1.0000,295.3000,305.7000,1168.0000,3.8000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,2500.7500,298.3000,308.8000,1423.0000,33.2000,53.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,5000.5000,300.1000,310.1000,1503.0000,40.1000,108.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
75%,7500.2500,301.5000,311.1000,1612.0000,46.8000,162.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
max,10000.0000,304.5000,313.8000,2886.0000,76.6000,253.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


From the above we can see that there is no null values are present inside the data as well as Air Temperature,Process temperature,Rtational speed and Torque are the continuous form of data and apart from that all are the categorical data.

In [24]:
df.rename(columns={"Air temperature [K]":"Air_Temp","Process temperature [K]":"Process_Temp","Rotational speed [rpm]":"Speed","Torque [Nm]":"Torque","Tool wear [min]":"Tool_wear"},inplace=True)

In [25]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air_Temp', 'Process_Temp', 'Speed',
       'Torque', 'Tool_wear', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [30]:
report = ProfileReport(df)

In [32]:
report.to_widgets()

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

- There is high correlation inbetween speed and Torque.

In [33]:
Save_report = report.to_file('Analysis.html')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
df['Type'].value_counts()

L    6000
M    2997
H    1003
Name: Type, dtype: int64

In [44]:
dic = {"L":0,"M":1,"H":2}
df['Type'] = df['Type'].map(dic)

In [46]:
df.drop(columns=['Product ID'],inplace=True)

In [53]:
df.drop(columns=['UDI'],inplace=True)

In [54]:
x = df.drop(columns=['Air_Temp'])
y = df['Air_Temp']

### Model Building.

In [55]:
model = LinearRegression()
model.fit(x,y)

LinearRegression()

In [56]:
model.intercept_

-64.17758531313257

In [57]:
model.coef_

array([-1.06613497e-02,  1.17371089e+00,  1.89769970e-04,  2.67025130e-04,
        8.45446054e-05, -9.18495802e-02,  2.04781243e-01,  1.78006185e+00,
        1.43330480e-01, -4.30601818e-02, -5.70538573e-02])

In [60]:
prediction = model.predict([[2,200,305,50,1000,19,1,1,1,1,1]])
prediction

array([170.9819629])

In [62]:
model.score(x,y)

0.775678768404829

In [63]:
import pickle

In [64]:
save_model = pickle.dump(model,open('demo.h5',mode='wb'))

In [66]:
load_model = pickle.load(open('demo.h5',mode='rb'))

In [67]:
load_model.predict([[2,200,305,50,1000,19,0,0,0,0,0]])

array([168.95390336])

### Thank You !!